# SQL クエリ生成のために Bedrock モデルを呼び出す

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

## 導入

このノートでは、LLMを使って SQL クエリを生成し、売上データを分析する方法を紹介します。

Boto3 API を使用した Bedrock の Claude V2 モデルを使用する。

この例で使用されているプロンプトは、プロンプト以外のテキストの例を提供していないため、Zero-Shot プロンプトと呼ばれています。

**注意:** *このノートブックは AWS 環境内外でも実行可能です*

#### コンテキスト
Amazon Bedrock の SQL コード生成機能を実証するために、Amazon Bedrock API 経由で Boto3 クライアントを利用します。利用可能な様々な構成と、単純な入力がどのように望ましい出力へつながるかを実演します。

#### パターン
Amazon Bedrock API に、タスク、命令、入力からなる入力を提供するだけで、追加の例を提供することなく、モデルが出力を生成します。ここでの目的は、強力な LLM がいかに簡単に手元のタスクを理解し、説得力のある出力を生成するかを示すことです。

![](./images/sql-query-generation.png)

#### ユースケース
販売データを分析するためにSQLクエリーを生成し、トレンド、上位商品、平均売上に焦点を当てるというユースケースを考えてみましょう。

#### ペルソナ
Maya は AnyCompany のビジネスアナリストで、主に販売と在庫データを担当しています。彼女はスプレッドシート分析からデータ駆動型分析に移行しており、SQL を使用して特定のデータ・ポイントを効率的に取得したいと考えています。彼女は LLM を使用して分析用の SQL クエリを生成したいと考えています。

#### 実装
このユースケースを実現するために、このノートブックでは SQL クエリを生成する方法を紹介します。Amazon Bedrock API と Boto3 クライアントを使用して、Anthropic Claude v2 モデルを使用します。

## セットアップ

このノートブックの残りの部分を実行する前に、以下のセルを実行して（必要なライブラリーがインストールされていることを確認し）Bedrock に接続する必要があります。

セットアップの仕組みや ⚠️ **変更が必要かどうかの詳細** については [Bedrock boto3 setup ノートブック](../00_Intro/bedrock_boto3_setup.ipynb) を参照してください。

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.309

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## SQLクエリの生成

上で説明したユースケースに従って、Amazon Bedrock サービスが SQL クエリを生成するための入力を用意しましょう。

In [45]:
# SQLクエリを生成するプロンプトを作成する
prompt_data = """

Human: AnyCompany has a database with a table named sales_data containing sales records. The table has following columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Can you generate SQL queries for the below: 
- Identify the top 5 best selling products by total sales for the year 2023
- Calculate the monthly average sales for the year 2023

Assistant:
"""


まずは Anthorpic Claude v2 モデルを使ってみましょう。

In [46]:
# Claude - Body 文
body = json.dumps({
                    "prompt": prompt_data,
                    "max_tokens_to_sample":4096,
                    "temperature":0.5,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 

#### Bedrock の Claude

まず、先に作成したプロンプトに基づき、モデルがどのように出力を生成するかを調べましょう。

##### 出力を生成する

In [ ]:
modelId = 'anthropic.claude-v2' # モデルプロバイダーによる切り替えが可能です
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

### 高度な例
#### SQL で病院の患者管理システムを理解する

In [48]:
# プロンプトを作成する
prompt_sql_data = """

Human: You're provided with a database schema representing any hospital's patient management system.
The system holds records about patients, their prescriptions, doctors, and the medications prescribed.

Here's the schema:

```sql
CREATE TABLE Patients (
    PatientID int,
    FirstName varchar(50),
    LastName varchar(50),
    DateOfBirth datetime,
    Gender varchar(10),
    PRIMARY KEY (PatientID)
);

CREATE TABLE Doctors (
    DoctorID int,
    FirstName varchar(50),
    LastName varchar(50),
    Specialization varchar(50),
    PRIMARY KEY (DoctorID)
);

CREATE TABLE Prescriptions (
    PrescriptionID int,
    PatientID int,
    DoctorID int,
    DateIssued datetime,
    PRIMARY KEY (PrescriptionID)
);

CREATE TABLE Medications (
    MedicationID int,
    MedicationName varchar(50),
    Dosage varchar(50),
    PRIMARY KEY (MedicationID)
);

CREATE TABLE PrescriptionDetails (
    PrescriptionDetailID int,
    PrescriptionID int,
    MedicationID int,
    Quantity int,
    PRIMARY KEY (PrescriptionDetailID)
);
```

Write a SQL query that fetches all the patients who were prescribed more than 5 different medications on 2023-04-01.

Assistant:
"""


In [49]:
# Claude - Body 文
body = json.dumps({
                    "prompt": prompt_sql_data,
                    "max_tokens_to_sample":4096,
                    "temperature":0.5,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 

In [ ]:
modelId = 'anthropic.claude-v2' # モデルプロバイダーによる切り替えが可能です
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

## 結論
 `boto3` SDK 経由で Amazon Bedrock API を触ってみました。 またこの API を使用して、販売データを分析するために SQL クエリを生成する使用例も確認しました。

### 発展要素
- このノートブックを使って、Amazon Bedrock で利用可能なさまざまなモデル（Anthropic Claude や AI21 Labs Jurassic モデルなど）を試してみてください。
- より良いアウトプットを得るために、さまざまな迅速なプロンプトエンジニアリング手法を適用してみてください

## ありがとうございます